In [3]:
!git clone https://github.com/Tingmi88/llm.git

Cloning into 'llm'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 19 (delta 1), reused 18 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 1004.33 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
!ls

llm  sample_data


In [5]:
%cd llm

/content/llm


In [6]:
!ls

data  example.ipynb  README.md	requirements.txt


In [7]:
!pip install llama-index-llms-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [9]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [11]:
!pip install llama-index-tools-google

In [12]:
import os
import getpass

from llama_index.llms.groq import Groq
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage)

from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from llama_index.tools.google import GoogleSearchToolSpec

### refererence:

Introduction to LlamaIndex with Python (2024) videos 1-3: https://www.youtube.com/watch?v=cCyYGYyCka4

https://docs.llamaindex.ai/en/stable/use_cases/agents/

tools:

https://llamahub.ai/?tab=tools

https://llamahub.ai/l/tools/llama-index-tools-google?from=tools

### 1. set up the embedding model: from hf

In [13]:
Settings.embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 2. set up the llm (free :))

In [14]:
os.environ['GROQ_API_KEY'] = getpass.getpass("GROQ API KEY:")

GROQ API KEY: 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··········


In [15]:
# configure the LLM for querying
Settings.llm = Groq(model="llama3-70b-8192")

### 3. create and store the index

In [17]:
!pip install llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.9 MB/s eta 0:00:00


In [18]:
# Define the directory for storing the index
PERSIST_DIR = "./storage"

# Load or create the index
if not os.path.exists(PERSIST_DIR):
    # Load the documents and create the index, all the documents are saved in the data folder
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)

    # Store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # Load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

### 4. create a query engine

In [19]:
query_engine = index.as_query_engine()

### 5. define a booksearch tool (we can add movie search tool, spotify album tool, etc)

In [20]:
booksearch_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="best_books",
    description="A RAG engine with some information of popular books."
)

### 6. define a search tool

In [21]:
# # Import and initialize our tool spec
from llama_index.core.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec
from llama_index.tools.google import GoogleSearchToolSpec

# reference: https://llamahub.ai/l/tools/llama-index-tools-google?from=tools
google_spec = GoogleSearchToolSpec(key="your-key", engine="your-engine-id")

tools = LoadAndSearchToolSpec.from_defaults(
    google_spec.to_tool_list()[0]
).to_tool_list()

all_tools = tools + [booksearch_tool]

# Initialize the ReAct Agent
agent = ReActAgent.from_tools(all_tools, verbose=True)

### 7. Run query

#### From the provided database

In [22]:
response1 = agent.chat("What is the best book to recommend to a 7 year old girl, and why?")

> Running step bf8f0cd7-e34d-47e0-bbc2-19ee02bd9a0b. Step input: What is the best book to recommend to a 7 year old girl, and why?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: best_books
Action Input: {'input': 'popular books for 7 year old girls'}
Observation: Based on the provided data, it appears that popular books for 7-year-old girls might include "Snow White and the Seven Dwarfs" by Walt Disney Company, "The Wimpy Kid Movie Diary" by Jeff Kinney, and possibly "Vanishing Girls" by Lauren Oliver. These books seem to be geared towards a younger audience and have received high ratings.
> Running step 480e3533-9195-4f19-b9c1-524a9ae7a46b. Step input: None
Thought: The observation provides some popular book options for a 7-year-old girl, but I'd like to get more information about these books to make a well-informed recommendation.
Action: google_search
Action Input: {'query': 'reviews of Snow White and the Seven Dwa

ValueError: Reached max iterations.

#### from google search

In [23]:
response2 = agent.chat("What're the best things to do in Orland, FL?")

> Running step 4afb1a48-901f-434e-b54e-2ef37b8acaf7. Step input: What're the best things to do in Orland, FL?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: google_search
Action Input: {'query': 'things to do in Orlando FL'}
Observation: Content loaded! You can now search the information using read_google_search
> Running step 9368f4bb-9bbc-4eb1-9561-a8dfbe894f93. Step input: None
Thought: I have the search results from Google, now I need to extract the relevant information.
Action: read_google_search
Action Input: {'query': 'what are the top attractions in Orlando'}
Observation: There is no information about Orlando or its attractions in the provided context. The context appears to be an error message related to an invalid API key. Therefore, it is not possible to provide an answer to the query based on the given context.
> Running step d7029173-09cf-4796-aec0-02f6856b09b6. Step input: None
Thought: It seems like the

ValueError: Reached max iterations.